In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
import os

In [3]:
# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set Pandas display options to see all columns clearly in console
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [4]:
print(os.getcwd())

C:\Users\pengu\CPP\DSAI\Flight Price Prediction


In [5]:
os.listdir("..")

['datathon', 'Flight Price Prediction', 'ml_workshop']

In [6]:
df = pd.read_csv("flight-price-prediction/Clean_Dataset.csv")
df

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...
300148,300148,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,69265
300149,300149,Vistara,UK-826,Chennai,Afternoon,one,Night,Hyderabad,Business,10.42,49,77105
300150,300150,Vistara,UK-832,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,300151,Vistara,UK-828,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585


In [7]:
# Removing high cardinality column 
# Removing index artifacts

drop_columns = ['flight', 'Unnamed: 0']

df.drop(columns=drop_columns, axis=1, inplace=True)

print(f'Columns dropped: {drop_columns}')

Columns dropped: ['flight', 'Unnamed: 0']


In [8]:
df.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [9]:
# Rename columns
df.columns = ["Airline", "Source", "Departure Time", "Stops", "Arrival Time", "Destination", "Class", "Duration", "Days Left", "Price"]
print(df.columns)

Index(['Airline', 'Source', 'Departure Time', 'Stops', 'Arrival Time', 'Destination', 'Class', 'Duration', 'Days Left', 'Price'], dtype='object')


In [10]:
# Value Counts
for col in df.columns:
    print(f"\n--- {col} ---")
    print(df[col].value_counts())


--- Airline ---
Airline
Vistara      127859
Air_India     80892
Indigo        43120
GO_FIRST      23173
AirAsia       16098
SpiceJet       9011
Name: count, dtype: int64

--- Source ---
Source
Delhi        61343
Mumbai       60896
Bangalore    52061
Kolkata      46347
Hyderabad    40806
Chennai      38700
Name: count, dtype: int64

--- Departure Time ---
Departure Time
Morning          71146
Early_Morning    66790
Evening          65102
Night            48015
Afternoon        47794
Late_Night        1306
Name: count, dtype: int64

--- Stops ---
Stops
one            250863
zero            36004
two_or_more     13286
Name: count, dtype: int64

--- Arrival Time ---
Arrival Time
Night            91538
Evening          78323
Morning          62735
Afternoon        38139
Early_Morning    15417
Late_Night       14001
Name: count, dtype: int64

--- Destination ---
Destination
Mumbai       59097
Delhi        57360
Bangalore    51068
Kolkata      49534
Hyderabad    42726
Chennai      40368
Name

In [11]:
# Convert Duration to minutes and rounding to nearest integer

df["Duration"] = (df["Duration"] * 60).round().astype(int)

In [28]:
df['Duration'].describe()

count    300153.000000
mean        733.259055
std         431.518381
min          50.000000
25%         410.000000
50%         675.000000
75%         970.000000
max        2990.000000
Name: Duration, dtype: float64

In [30]:
df['Price'].describe()

count    300153.000000
mean      20889.660523
std       22697.767366
min        1105.000000
25%        4783.000000
50%        7425.000000
75%       42521.000000
max      123071.000000
Name: Price, dtype: float64

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Airline         300153 non-null  object
 1   Source          300153 non-null  object
 2   Departure Time  300153 non-null  object
 3   Stops           300153 non-null  object
 4   Arrival Time    300153 non-null  object
 5   Destination     300153 non-null  object
 6   Class           300153 non-null  object
 7   Duration        300153 non-null  int32 
 8   Days Left       300153 non-null  int64 
 9   Price           300153 non-null  int64 
dtypes: int32(1), int64(2), object(7)
memory usage: 21.8+ MB


In [34]:
df.sample(10)

,Airline,Source,Departure Time,Stops,Arrival Time,Destination,Class,Duration,Days Left,Price
238568,Air_India,Mumbai,Early_Morning,one,Night,Hyderabad,Business,800,10,50868
299709,Vistara,Chennai,Night,one,Afternoon,Hyderabad,Business,945,40,41064
79491,AirAsia,Mumbai,Early_Morning,one,Afternoon,Chennai,Economy,450,18,1890
214754,Vistara,Delhi,Evening,one,Early_Morning,Bangalore,Business,770,35,57017
73369,Air_India,Mumbai,Night,one,Morning,Hyderabad,Economy,780,23,4173
139639,Vistara,Kolkata,Afternoon,one,Night,Bangalore,Economy,440,30,8321
39103,Vistara,Delhi,Evening,one,Morning,Chennai,Economy,965,26,4979
33410,Vistara,Delhi,Afternoon,one,Early_Morning,Hyderabad,Economy,980,36,5772
227464,Vistara,Mumbai,Early_Morning,one,Evening,Delhi,Business,630,26,36565
22781,Air_India,Delhi,Evening,one,Evening,Kolkata,Economy,1630,17,4559


In [36]:
df.to_csv('flight-price-prediction/clean_data_v2', index=False)
print('Saved cleaned data inside data folder')

Saved cleaned data inside data folder
